# 랜덤포레스트(Random Forest)

In [9]:
#데이터 불러오기.
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine-date')
date = wine[['alcohol', 'sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(date, target, test_size=0.2, random_state=42)

print(wine.head(3))

   alcohol  sugar    pH  class
0      9.4    1.9  3.51    0.0
1      9.8    2.6  3.20    0.0
2      9.8    2.3  3.26    0.0


In [11]:
#cross_validate()함수를 사용해 교차검증 수행하기.
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
sc = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(sc['train_score']), np.mean(sc['test_score']))

0.9973541965122431 0.8905151032797809


**랜덤 포레스트는 결정트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공한다.**   
**랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다.**

In [13]:
#랜덤포레스트의 특성의 중요도 평가하기.(하나의 특성에 과도하게 집중되지 않는다.)
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [14]:
#자체적으로 모델을 평가하는 점수를 얻을 수 있다.
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리(Extra Trees)
- 랜덤 포레스트와 비슷하지만, 부트스트랩 샘플을 사용하지 않는다.
- 노드를 분할 할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.

In [18]:
#엑스트라 트리.
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
sc = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(sc['train_score']), np.mean(sc['test_score']))

0.9974503966084433 0.8887848893166506


In [16]:
#특성의 중요도
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그레디언트 부스팅(Gradient boosting)
- 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블한다.
- 기본적으로 깊이가 3인 결정트리를 100개 사용한다.
- 과대적합에 강하고 일반적으로 높은 일반화 성능을 가진다.

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
sc = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(sc['train_score']), np.mean(sc['test_score']))
#결과가 좋다.

0.8881086892152563 0.8720430147331015


In [22]:
#결정트리의 수를 500개로 늘려보기 학습률도 높이기.
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
sc = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(sc['train_score']), np.mean(sc['test_score']))

0.9464595437171814 0.8780082549788999


In [23]:
#특성 중요도(그레디언트 부스팅이 랜덤포레스트보다 일부특성에 더 집중한다.
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# 히스토그램 기반 그레디언트 부스팅(Histogram-based Gradient Boosting)
- 그레디언트 부스팅의 속도와 성능을 개선하였음.
- 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘이다.
- 입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할 할 때 특성의 최적의 분할을 매우 빠르게 찾을 수 있다.
- 256개의 구간중에서 하나를 떼어 놓고 누란된 값을 위해 사용한다. 따라서 입력이 안된 특성이 있더라도 따로 전처리하지 않아도 된다.

In [25]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble  import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
sc = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(sc['train_score']), np.mean(sc['test_score']))

0.9321723946453317 0.8801241948619236


In [27]:
#특성의 중요도
hgb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [29]:
#성능확인
hgb.score(test_input, test_target)

0.8723076923076923

# XGBoost, LightGBM 라이브러리

In [30]:
#XGBoost 라이브러리.
#tree_method 매개변수에 'hist'로 지정하면 히스토그램 기반 그레디언트 부스팅을 사용할 수 있다.
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
sc = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(sc['train_score']), np.mean(sc['test_score']))

C:\Users\User\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:59:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:59:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:59:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:59:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [33]:
#LightGBM 라이브러리.
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
sc = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(sc['train_score']), np.mean(sc['test_score']))

0.935828414851749 0.8801251203079884
